<div class="alert alert-block alert-info">
<h1> Case Study 1 </h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import plotly.express as px # for USA map
from mpl_toolkits import mplot3d # for 3D graph
%config InlineBackend.figure_format = 'retina'

import seaborn as sns # for heatmap
from sklearn.model_selection import train_test_split # to split the dataset
import statsmodels.api as sm # Linear Regression Approach 1
from sklearn.linear_model import LinearRegression # Linear Regression Approach 2
from sklearn.metrics import r2_score # to evaluate predictions


### 1. Dataset Pre-processing

In [ ]:
df = pd.read_csv('loans_full_schema.csv')
df.head(10)

In [ ]:
df.info()

#### 1.1 Remove NULL values

Let's see how many NULL values we have in our dataset.

In [ ]:
df.isnull().sum()

Unfortunately there are columns where most of values are NULLs. They are completely useless, so just remove those columns where more than 1% of the rows for that column contain a null value.

In [ ]:
df_cleaned = df[[label for label in df if df[label].isnull().sum() <= 0.01 * df.shape[0]]]

Let's see how it looks like now.

In [ ]:
df_cleaned.isnull().sum()

Ok, much better. Now we have to do something with those NULL values. We can:

* remove rows cointain NULL values,
* fill them with median or mode value,
* or use some imputation and try to predict their missing values.

Let's try the first option and see what will happen.

In [ ]:
df_cleaned = df_cleaned.dropna()
df_cleaned.shape[0] / df.shape[0]

It looks good, we removed less than 0.3% of rows. I think it's good enough and there's no point to do something more with that.

#### 1.2 Remove useless columns

Let's take a look on our cleaned dataset.

In [ ]:
df_cleaned.head(10)

As we can see, "issue_month" only includes three values: "Jan-2018", "Feb-2018", and "Mar-2018". The time does not vary much and its impact to the applicant's credit profile is negligible. Thus column "issue_month" can be dropped.

The same goes for "disbursement_method" which only includes two values: "Cash" and "DirectPay". The impact of these methods to the applicant's credit profile can also be neglected. Thus column "disbursement_method" can also be dropped.

Since "sub_grade" (a grade assigned to the loan applicant based on his credit profile which determines the interest rate) has fully included the information of "grade", column "grade" can be dropped as well. A detailed explanation of subgrade can be found here: https://www.lendingclub.com/public/rates-and-fees.action

Since the three major US credit bureaus no longer include tax liens on your credit reports, a tax lien is no longer able to affect your credit. Thus, column "tax_liens" can be dropped. Details can be found here: https://www.bankrate.com/finance/credit-cards/how-tax-liens-affect-your-credit-score/, 

In [ ]:
df_cleaned = df_cleaned.drop(["grade", "issue_month", "disbursement_method", "tax_liens"], axis=1)
df_cleaned.head(10)

#### 1.3 Remove columns including values with insignificant frequencies and outlier rows

In [ ]:
for label in list(df_cleaned):
    if len(df_cleaned[label].unique()) < 20:
        print(df_cleaned[label].value_counts())
        print("\n")

We can see that features "current_accounts_delinq" and "num_accounts_30d_past_due" have only two possible values: 0 and 1, but with only 1 occurrences of 1 (less than 1%), so definitely they are insignificant.

The same goes for "num_collections_last_12m" feature (non-zero values have only 1.3% frequency).

Therefore we can drop these three columns.

Since >1 values in "num_historical_failed_to_pay", >1 values in "public_record_bankrupt", >2 values in "delinq_2y", >6 values in "num_mort_accounts" and values other than "Current" and "Fully Paid" in "loan_status" have very low frequency (less than 2%), we can remove these outliers. The same goes with "moving", "vacation" and "renewable energy" values in "loan_purpose".


In [ ]:
df_cleaned = df_cleaned.drop(["current_accounts_delinq", "num_accounts_30d_past_due", "num_collections_last_12m"], axis=1)
df_cleaned = df_cleaned[df_cleaned['num_historical_failed_to_pay'] <= 2]
df_cleaned = df_cleaned[df_cleaned['public_record_bankrupt'] <= 1]
df_cleaned = df_cleaned[df_cleaned['delinq_2y'] <= 3]
df_cleaned = df_cleaned[df_cleaned['num_mort_accounts'] <= 7]
df_cleaned = df_cleaned[(df_cleaned['loan_status'] == "Current") | (df_cleaned['loan_status'] == "Fully Paid")]
df_cleaned = df_cleaned[(df_cleaned['loan_purpose'] == "debt_consolidation") | (df_cleaned['loan_purpose'] == "credit_card")
                       | (df_cleaned['loan_purpose'] == "other") | (df_cleaned['loan_purpose'] == "home_improvement")
                       | (df_cleaned['loan_purpose'] == "major_purchase") | (df_cleaned['loan_purpose'] == "medical")
                       | (df_cleaned['loan_purpose'] == "house") | (df_cleaned['loan_purpose'] == "car")
                       | (df_cleaned['loan_purpose'] == "small_business") | (df_cleaned['loan_purpose'] == "moving")]
df_cleaned.head(10)

In [ ]:
df_cleaned.shape[0] / df.shape[0]

So the removed records are no more than 5%, which means the cleaned dataset can still represent the original dataset well.

#### 1.4 Categorical features

To use any machine learning model we have to have only numerical data. So, let's do something with our non-numerical features.

In [ ]:
df_cleaned.select_dtypes(include=["object"]).head()

Since each grade (A, B, C, D, E, F) and the associated subgrade have its own sequence, we can map them into integers.

In [ ]:
def subgrade_mapping(lst: list) -> dict:
    mapping = {}
    for subgrade in lst:
        
        subgrade = subgrade.strip()
        letter = subgrade[0]
        num = int(subgrade[1])
        if letter == 'A':
            m = 0
        elif letter == 'B':
            m = 1
        elif letter == 'C':
            m = 2
        elif letter == 'D':
            m = 3
        elif letter == 'E':
            m = 4
        elif letter == 'F':
            m = 5
        elif letter == 'G':
            m = 6
        else:
            m = np.nan
        
        mapping[subgrade] = m*5+num
    return mapping

df_cleaned["sub_grade"] = df_cleaned["sub_grade"].map(subgrade_mapping(df_cleaned["sub_grade"].unique()))

To show the impact of states, let's plot the average interest rate by states on the US map:

In [ ]:
df_avg_by_state = df_cleaned.groupby('state').mean()
df_avg_by_state = df_avg_by_state.reset_index()

fig = px.choropleth(df_avg_by_state,  # Input Pandas DataFrame
                    locations="state",  # DataFrame column with locations
                    color="interest_rate",  # DataFrame column with color values
                    hover_name="state", # DataFrame column hover info
                    locationmode = 'USA-states',# Set to plot as US States
                    color_continuous_scale="purp") #set color scale
fig.update_layout(
    title_text = 'Interest Rate by States', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()

As we can see from the plot above, the average interest rate has no significant difference among different states, and there is also no significant trend on the map. Thus, we decide to drop this feature otherwise too many dummy indicators will be created.

In [ ]:
df_cleaned = df_cleaned.drop(["state"], axis=1)

The rest of the categorical features can be either mapped to integers, or to dummy indicators.

In [ ]:
#df_cleaned["initial_list_status"] = df_cleaned["initial_list_status"].map({"fractional": 1, "whole": 0})
df_cleaned["application_type"] = df_cleaned["application_type"].map({"individual": 1, "joint": 0})
df_cleaned["loan_status"] = df_cleaned["loan_status"].map({"Current": 1, "Fully Paid": 0})
df_cleaned["verified_income"] = df_cleaned["verified_income"].map({"Not Verified": 2, "Source Verified": 1, "Verified": 0})
df_cleaned["homeownership"] = df_cleaned["homeownership"].map({"RENT": 2, "MORTGAGE": 1, "OWN": 0})
df_cleaned = pd.get_dummies(df_cleaned, columns=list(df_cleaned.select_dtypes(include=["object"])))

We can calculate the correlation matrix for the cleaned dataset, and then plot the values int the heatmap:

In [ ]:
C = df_cleaned.corr()
C

In [ ]:
# Visualizing the data using heatmap
fig, ax = plt.subplots(figsize = (15,10))
sns.heatmap(df_cleaned.corr(), cmap="YlGnBu", annot = True)
plt.show()

In [ ]:
C['interest_rate']

As we can see from the graph and table above, the most relevant variable is sub_grade (0.993) and paid_interest(0.522)
We can plot them as scatter plots to see the data distribution!

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
ax.scatter(df_cleaned['sub_grade'], df_cleaned['interest_rate'], color='blue', alpha=0.3)

ax.set_xlabel('Subgrade', fontsize=12)
ax.set_ylabel('Interest Rate (%)', fontsize=12)
ax.set_title("Interest Rate by Subgrade", size=14)
# Set numerical y axis
ax.yaxis.set_label_coords(-0.08, 0.5)
# Set categorical ticks for x axis
ax.set_xticks(np.arange(7)*5)
ax.set_xticklabels(['A','B','C','D', 'E', 'F', 'G'], size=12)
#ax.tick_params(axis='x', length=0)    #Hide ticks in x axis
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(.5)
ax.spines['bottom'].set_linewidth(.5)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
ax.scatter(df_cleaned['paid_interest'], df_cleaned['interest_rate'], color='blue', alpha=0.1)

ax.set_xlabel('Paid Interest', fontsize=12)
ax.set_ylabel('Interest Rate (%)', fontsize=12)
ax.set_title("Interest Rate by Paid Interest", size=14)
# Set numerical y axis
ax.yaxis.set_label_coords(-0.08, 0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(.5)
ax.spines['bottom'].set_linewidth(.5)


plt.show()

We can even plot them into 3D graph:

In [ ]:

# Creating figure
fig = plt.figure(figsize = (30, 10))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(df_cleaned['paid_interest'], df_cleaned['sub_grade'], df_cleaned['interest_rate'],
             color = 'blue', s=3,alpha=0.3)

# Set categorical ticks for y axis
ax.set_yticks(np.arange(7)*5)
ax.set_yticklabels(['A','B','C','D', 'E', 'F', 'G'], size=12)

ax.set_xlabel('Paid Interest')
ax.set_ylabel('Subgrade')
ax.set_zlabel('Interest Rate (%)')
plt.title("Interest Rate Distribution with Subgrade and Paid Interest")
 
# show plot
plt.show()

As we can see from the graphs above, subgrade has a strong linear correlation to interest rate; paid interest mainly determine the lower limit of the interest rate, but has no restraint to the upper limit.

### 2. Modeling Approach 1: Linear Regression Using `statsmodel`

#### 2.1 Perform Linear Regression

In [ ]:
X = df_cleaned['sub_grade']
y = df_cleaned['interest_rate']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7,
                                                    test_size = 0.3, random_state = 100)

In [ ]:
'''
By default, the statsmodel library fits a line that passes through the origin.
But if we observe the simple linear regression equation y = c + mX,
it has an intercept value as c.
So, to have an intercept, we need to add the add_constant attribute manually.
'''
X_train_sm = sm.add_constant(X_train)
# Fitting the resgression line using Ordinary Least Square method
lr = sm.OLS(y_train, X_train_sm).fit()

# Printing the parameters
lr.params

In [ ]:
# Performing a summary to list out all the different parameters of the regression line fitted
lr.summary()

<div class="alert alert-block alert-success">
<h3> Discussions: </h3>

* The coefficient for `sub_grade` is 0.8457, and its corresponding p-value is very low (almost 0). That means the coefficient is statistically significant.
* R-squared value is 0.986, which means that 98.6% of the interest rate variance can be explained by the subgrade column using this line.
* Prob (F-statistic) has a very low p-value, practically zero, which gives us that the model fit is statistically significant.

Since the fit is significant, let's go ahead and visualize how well the straight-line fits the scatter plot between `sub_grade` and `interest_rate` columns.
        
</div>

#### 2.2 Visualization of the Regression Line

From the parameters shown above, we have obtained the values of the intercept and the slope of the straight line. The equation of the line is

$interest$ $rate=3.8372+0.8457*subgrade $

In [ ]:
# Visualizing the regression line
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(X_train, y_train)
ax.plot(X_train, 3.8372 + 0.8457*X_train, 'r')
ax.set_title('Best-fit regression line', fontsize = 15)
ax.set_xlabel('Subgrade', fontsize = 12)
ax.set_ylabel('Interest Rate(%)', fontsize = 12)
plt.show()

#### 2.3 Residual Analysis

One of the major assumptions of the linear regression model is the error terms are normally distributed.

$Error=y-\hat{y}$

where $y$ is the actual y value, and $\hat{y}$ is the predicted y value.

Now from the dataset, we have to predict the y value from the training dataset of X using the predict attribute. After that, we’ll create the error terms(Residuals) from the predicted data.

In [ ]:
# Predicting y_value using traingn data of X
y_train_pred = lr.predict(X_train_sm)

# Creating residuals from the y_train data and predicted y_data
res = (y_train - y_train_pred)

Now, let’s plot the histogram of the residuals and see whether it looks like normal distribution or not.

In [ ]:
# Plotting the histogram using the residual values
fig, ax = plt.subplots(figsize=(6,4))
n, bins, patches = ax.hist(res, bins=25)
ax.set_title('Error Terms', fontsize = 15)
ax.set_xlabel('y_train - y_train_pred', fontsize = 15)
ax.set_xlim(-4, 4)
plt.show()

<div class="alert alert-block alert-success">

As we can see, the residuals are following the normal distribution graph with a mean 0.
        
</div>

Now, make sure that the residuals are not following any specific pattern.

In [ ]:
# Looking for any patterns in the residuals
fig, ax = plt.subplots(figsize=(6,5))
ax.scatter(X_train, res)
ax.set_ylim(-4, 4)
plt.show()

<div class="alert alert-block alert-success">

Since the Residuals follow a normal distribution and do not follow any specific pattern, we can use the linear regression model we have built to evaluate test data.
        
</div>

#### 2.4. Predictions on the Test Data and Evaluations

Now that we have fitted the regression line on our train dataset, we can make some predictions to the test data. Similar to the training dataset, we have to `add_constant` to the test data and predict the y values using the `predict` attribute present in the `statsmodel`.

In [ ]:
# Adding a constant to X_test
X_test_sm = sm.add_constant(X_test)

# Predicting the y values corresponding to X_test_sm
y_test_pred = lr.predict(X_test_sm)

# Printing the first 15 predicted values
y_test_pred

Now, let’s calculate the `R2` value for the above-predicted y-values. We can do that by merely importing the `r2_score` library from `sklearn.metrics` package.

In [ ]:
# Checking the R-squared value
r_squared = r2_score(y_test, y_test_pred)
r_squared

<div class="alert alert-block alert-success">

Since the R2 value on training data is 0.986, the R2 value on test data is within 5% of the R2 value on training data. In this case we can conclude that the model is pretty stable. This means, what the model has learned on the trainign set can generalize on the unseen test set.
</div>

Let's visualize the line on the test data.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(X_test, y_test)
ax.plot(X_test, y_test_pred, 'r')
ax.set_title('Best-fit regression line', fontsize = 15)
ax.set_xlabel('Subgrade', fontsize = 12)
ax.set_ylabel('Interest Rate(%)', fontsize = 12)
plt.show()

### 3. Modeling Approach 2: Linear Regression Using `sklearn`

#### 3.1 Perform Linear Regression

In [ ]:
X_train_lm, X_test_lm, y_train_lm, y_test_lm = train_test_split(X, y, train_size = 0.7, 
                                                                test_size = 0.3, random_state = 100)


For simple linear regression, we need to add a column to perform the regression fit properly.

In [ ]:
# Shape of the train set without adding column
print("The shape of X_train before adding a column is ", X_train_lm.shape)

# Adding additional column to the train and test data
X_train_lm = X_train_lm.values.reshape(-1,1)
X_test_lm = X_test_lm.values.reshape(-1,1)

print("The shape of X_train after adding a column is ", X_train_lm.shape)
print("The shape of X_test after adding a column is ", X_test_lm.shape)

Now we can conduct the linear regression using `sklearn.linear_model`

In [ ]:
# Creating an object of Linear Regression
lm = LinearRegression()

# Fit the model using .fit() method
lm.fit(X_train_lm, y_train_lm)

# Intercept value
print("Intercept :",lm.intercept_)

# Slope value
print('Slope :',lm.coef_)

The straight-line equation we get for the above values is,

$interest$ $rate=3.8372+0.8467*subgrade$

If we observe, the equation we got here is the same as the one we got in the `statsmodel`.

#### 3.2 Predictions on the Test Data and Evaluations¶

In [ ]:
# Making Predictions of y_value
y_train_pred = lm.predict(X_train_lm)
y_test_pred = lm.predict(X_test_lm)

# Comparing the r2 value of both train and test data
print(r2_score(y_train,y_train_pred))
print(r2_score(y_test,y_test_pred))

<div class="alert alert-block alert-success">

Same as the statesmodel, the R² value on test data is within 5% of the R² value on training data. We can apply the model to the unseen test set in the future.
</div>

### 4. Conclusions

<div class="alert alert-block alert-success">

* Data cleaning is performed to the original dataset. Unrelevant columns and outlier rows are removed.

* After calculating the correlation coefficients, it is found that `sub_grade` is able to dominate the target variable `interest_rate`.
    
* In this case, we can simply use numerized `sub_grade` value to build up linear regression model in `statsmodel` and `sklearn`. Both of the model predictions have shown good agreement with the actual values.
    
* Model evaluations are made to both modeling approaches and r2 score is used to prove the model has enough robustness and stability.
</div>



### 5. Reference

https://towardsdatascience.com/simple-linear-regression-model-using-python-machine-learning-eab7924d18b4